In [28]:
# basic
import numpy as np
import pandas as pd
from itertools import product
# plot
import seaborn as sns
import matplotlib.pyplot as plt
# preprocess
from sklearn.preprocessing import LabelEncoder
# model
import lightgbm as lgb

In [29]:
items = pd.read_csv('../data/input/items.csv')
shops = pd.read_csv('../data/input/shops.csv')
cats = pd.read_csv('../data/input/item_categories.csv')
train = pd.read_csv('../data/input/sales_train.csv')
# set index to ID to avoid droping it later
test  = pd.read_csv('../data/input/test.csv').set_index('ID')

# preprocess train

In [30]:
# processing outlier, negative value
train = train[train.item_cnt_day < 100000]
train = train[train.item_price < 1250]

median = train[(train.shop_id==32)&\
                     (train.item_id==2973)&\
                     (train.date_block_num==4)&\
                     (train.item_price>0)].item_price.median()
train.loc[train.item_price < 0, 'item_price'] = median

In [31]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [32]:
train['date'] = pd.to_datetime(train['date'], format="%d.%m.%Y")
train['year'] = train.date.dt.year
train['month'] = train.date.dt.month
train['day'] = train.date.dt.day

In [33]:
train.sample(10, random_state=0)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
2743911,2015-07-27,30,55,4260,499.0,1.0,2015,7,27
87181,2013-01-13,0,54,9615,349.0,1.0,2013,1,13
2832546,2015-09-14,32,42,15521,399.0,1.0,2015,9,14
1863961,2014-07-15,18,25,15235,799.0,1.0,2014,7,15
2748713,2015-07-18,30,50,21929,229.0,1.0,2015,7,18
288712,2013-03-09,2,19,21782,449.0,1.0,2013,3,9
74874,2013-01-01,0,54,22015,699.0,1.0,2013,1,1
2290918,2014-12-30,23,27,7085,699.0,1.0,2014,12,30
994493,2013-10-28,9,57,19356,249.0,1.0,2013,10,28
2621641,2015-05-09,28,4,12524,449.0,1.0,2015,5,9


# preprocess shops

In [34]:
# create column:shop_city_name
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city_name'] = shops['shop_name'].apply(lambda x:x.split(' ')[0])
# proofreading shop_city_name
shops.loc[shops['city_name']=='!Якутск', \
         'city_name'] = 'Якутск'
# Encoding
shops['city_code'] = LabelEncoder().fit_transform(shops['city_name'])
shops = shops[['shop_id','city_code']]

In [35]:
shops.sample(10, random_state=0)

,shop_id,city_code
26,26,13
35,35,15
59,59,30
28,28,13
11,11,6
2,2,0
34,34,15
58,58,29
40,40,18
22,22,13


# preprocess cats

In [36]:
cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])

# sub type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

# How deal with Чистые носители (штучные) & blank media (piece) ???

In [37]:
cats.sample(10, random_state=0)

,item_category_id,type_code,subtype_code
30,30,8,55
40,40,11,4
43,43,12,21
50,50,12,46
22,22,5,14
54,54,12,61
2,2,1,10
56,56,13,3
26,26,6,61
8,8,2,26


In [38]:
items.drop(['item_name'], axis=1, inplace=True)

# Create Monthly Sale

In [39]:
# test data is (shop_id) x (item_id)
# so create train-matrix:(shop_id) x (item_id)

matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())),\
                           dtype='int16'))

matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [40]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id
141119,0,58,2553
1927002,5,7,14219
3784633,11,24,16514
1937270,5,25,17300
6394530,19,22,13725
1389128,4,27,3909
3745893,10,52,7767
5873298,17,44,8331
3634783,10,22,17174
2722074,7,4,7396


In [41]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

group = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day':['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month'].fillna(0).clip(0, 20).astype(np.float16))

In [42]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month
321192,0,58,2553,0.0
1764657,5,7,14219,0.0
3896236,11,24,16514,0.0
1867821,5,25,17300,0.0
6306381,19,22,13725,0.0
1522508,4,27,3909,0.0
3705006,10,52,7767,0.0
5822987,17,44,8331,0.0
3554914,10,22,17174,0.0
2454039,7,4,7396,0.0


# Test set

In [43]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [44]:
matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)

In [45]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month
7120040,22,21,2693,0.0
3300657,9,37,22021,0.0
8237891,26,47,6222,0.0
3659678,10,44,2407,1.0
6053335,18,29,21840,0.0
7725711,24,38,6888,1.0
76270,0,15,6579,0.0
8720344,29,11,6766,0.0
3813884,11,11,19726,0.0
4043462,11,51,2384,0.0


# Shops/Items/Cats

In [46]:
matrix = pd.merge(matrix, shops, on='shop_id', how='left')
matrix = pd.merge(matrix, items, on='item_id', how='left')
matrix = pd.merge(matrix, cats, on='item_category_id', how='left')
matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)

In [47]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code
7120040,22,21,2693,0.0,13,31,8,61
3300657,9,37,22021,0.0,16,37,11,1
8237891,26,47,6222,0.0,22,55,13,2
3659678,10,44,2407,1.0,20,30,8,55
6053335,18,29,21840,0.0,13,40,11,4
7725711,24,38,6888,1.0,17,55,13,2
76270,0,15,6579,0.0,9,25,5,18
8720344,29,11,6766,0.0,6,30,8,55
3813884,11,11,19726,0.0,6,40,11,4
4043462,11,51,2384,0.0,24,31,8,61


# Target Lags

In [48]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return df

In [49]:
matrix = lag_feature(df=matrix, lags=[1, 3, 6, 9, 12], col='item_cnt_month')
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_9,item_cnt_month_lag_12
7120040,22,21,2693,0.0,13,31,8,61,0.0,0.0,NaN,NaN,NaN
3300657,9,37,22021,0.0,16,37,11,1,0.0,0.0,0.0,NaN,NaN
8237891,26,47,6222,0.0,22,55,13,2,0.0,0.0,0.0,1.0,0.0
3659678,10,44,2407,1.0,20,30,8,55,NaN,NaN,NaN,NaN,NaN
6053335,18,29,21840,0.0,13,40,11,4,NaN,0.0,1.0,0.0,0.0
7725711,24,38,6888,1.0,17,55,13,2,2.0,0.0,NaN,NaN,NaN
76270,0,15,6579,0.0,9,25,5,18,NaN,NaN,NaN,NaN,NaN
8720344,29,11,6766,0.0,6,30,8,55,0.0,1.0,0.0,NaN,NaN
3813884,11,11,19726,0.0,6,40,11,4,0.0,0.0,0.0,1.0,NaN
4043462,11,51,2384,0.0,24,31,8,61,NaN,NaN,NaN,NaN,NaN


# Mean encoded features

In [50]:
group = matrix.groupby('date_block_num').agg({'item_cnt_month':['mean']})
group.columns = ['date_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(df=matrix, lags=[1], col='date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

In [51]:
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month':['mean']})
group.columns = ['date_item_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

In [52]:
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month':['mean']})
group.columns = ['date_shop_avg_itme_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id'], how='left')
matrix['date_shop_avg_itme_cnt'] = matrix['date_shop_avg_itme_cnt'].astype(np.float16)
matrix.drop(['date_shop_avg_itme_cnt'], axis=1, inplace=True)

In [55]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)

In [56]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [57]:
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

In [58]:
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix['date_shop_subtype_avg_item_cnt'] = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_subtype_avg_item_cnt')
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)

In [59]:
group = matrix.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'city_code'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

In [60]:
group = matrix.groupby(['date_block_num', 'item_id', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'city_code'], how='left')
matrix['date_item_city_avg_item_cnt'] = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_city_avg_item_cnt')
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

In [61]:
group = matrix.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'type_code'], how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)

In [62]:
group = matrix.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'subtype_code'], how='left')
matrix['date_subtype_avg_item_cnt'] = matrix['date_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_subtype_avg_item_cnt')
matrix.drop(['date_subtype_avg_item_cnt'], axis=1, inplace=True)

In [63]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_3,...,item_cnt_month_lag_12,date_avg_item_cnt_lag_1,date_cat_avg_item_cnt_lag_1,date_shop_cat_avg_item_cnt_lag_1,date_shop_type_avg_item_cnt_lag_1,date_shop_subtype_avg_item_cnt_lag_1,date_city_avg_item_cnt_lag_1,date_item_city_avg_item_cnt_lag_1,date_type_avg_item_cnt_lag_1,date_subtype_avg_item_cnt_lag_1
7120040,22,21,2693,0.0,13,31,8,61,0.0,0.0,...,NaN,0.246094,0.056793,0.000000,0.329102,0.000000,0.407715,0.0,0.452148,0.065918
3300657,9,37,22021,0.0,16,37,11,1,0.0,0.0,...,NaN,0.281006,0.198853,0.152588,0.094910,0.150146,0.153931,0.0,0.213379,0.191895
8237891,26,47,6222,0.0,22,55,13,2,0.0,0.0,...,0.0,0.261963,0.220215,0.215210,0.166138,0.215210,0.275635,0.0,0.185059,0.220215
3659678,10,44,2407,1.0,20,30,8,55,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6053335,18,29,21840,0.0,13,40,11,4,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7725711,24,38,6888,1.0,17,55,13,2,2.0,0.0,...,NaN,0.368408,0.345947,0.245239,0.181641,0.245239,0.280518,2.0,0.279297,0.345947
76270,0,15,6579,0.0,9,25,5,18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8720344,29,11,6766,0.0,6,30,8,55,0.0,1.0,...,NaN,0.232788,0.743164,0.366211,0.218628,0.366211,0.081726,0.0,0.369873,0.743164
3813884,11,11,19726,0.0,6,40,11,4,0.0,0.0,...,NaN,0.268799,0.244873,0.054779,0.047821,0.054779,0.103821,0.0,0.218384,0.244873
4043462,11,51,2384,0.0,24,31,8,61,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Trend Features

In [65]:
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1, 2, 3, 4, 5, 6]
matrix = lag_feature(df=matrix, lags=lags, col='date_item_avg_item_price')

In [66]:
matrix.sample(10, random_state=0)

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_3,...,date_type_avg_item_cnt_lag_1,date_subtype_avg_item_cnt_lag_1,item_avg_item_price,date_item_avg_item_price,date_item_avg_item_price_lag_1,date_item_avg_item_price_lag_2,date_item_avg_item_price_lag_3,date_item_avg_item_price_lag_4,date_item_avg_item_price_lag_5,date_item_avg_item_price_lag_6
7120040,22,21,2693,0.0,13,31,8,61,0.0,0.0,...,0.452148,0.065918,499.000,499.0,499.00,499.0000,499.00,499.0,NaN,NaN
3300657,9,37,22021,0.0,16,37,11,1,0.0,0.0,...,0.213379,0.191895,743.000,549.0,549.00,999.0000,958.50,770.5,999.0,978.500
8237891,26,47,6222,0.0,22,55,13,2,0.0,0.0,...,0.185059,0.220215,291.500,299.0,299.00,NaN,299.00,299.0,299.0,299.000
3659678,10,44,2407,1.0,20,30,8,55,NaN,NaN,...,NaN,NaN,236.750,294.0,NaN,NaN,NaN,NaN,NaN,NaN
6053335,18,29,21840,0.0,13,40,11,4,NaN,0.0,...,NaN,NaN,141.375,98.0,NaN,108.1875,149.00,149.0,149.0,139.875
7725711,24,38,6888,1.0,17,55,13,2,2.0,0.0,...,0.279297,0.345947,313.250,298.5,297.75,299.0000,299.00,NaN,NaN,NaN
76270,0,15,6579,0.0,9,25,5,18,NaN,NaN,...,NaN,NaN,513.000,519.0,NaN,NaN,NaN,NaN,NaN,NaN
8720344,29,11,6766,0.0,6,30,8,55,0.0,1.0,...,0.369873,0.743164,426.750,248.0,248.00,290.7500,327.75,397.5,448.0,537.000
3813884,11,11,19726,0.0,6,40,11,4,0.0,0.0,...,0.218384,0.244873,144.625,149.0,149.00,149.0000,149.00,149.0,149.0,149.000
4043462,11,51,2384,0.0,24,31,8,61,NaN,NaN,...,NaN,NaN,93.250,99.0,NaN,99.0000,NaN,NaN,NaN,NaN


In [ ]:
for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
    (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price'])\
    / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

# https://stackoverflow.com/questions/31828240/first-non-null-value-per-row-from-a-list-of-pandas-columns/31828559
# matrix['price_trend'] = matrix[['delta_price_lag_1','delta_price_lag_2','delta_price_lag_3']].bfill(axis=1).iloc[:, 0]
# Invalid dtype for backfill_2d [float16]

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)